In [2]:
# This cell is intentionally left empty to avoid import errors.
# If you need to load a model, consider using the imports and configurations from later cells in the notebook.

In [ ]:
"""
Load the model and tokenizer using the custom loader script.
"""
import sys
import torch
sys.path.append('..')
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from scripts.model_loader import load_model
model, tokenizer = load_model()

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Fix for the deprecation warning
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

print("✓ torch imported successfully")
print("✓ transformers imported successfully")
print(f"Torch version: {torch.__version__}")

# Test the imports work
tokenizer = AutoTokenizer.from_pretrained("gpt2")
print("✓ AutoTokenizer works")
model = AutoModelForCausalLM.from_pretrained("gpt2")
print("✓ AutoModelForCausalLM works")
print("\n✓ All imports and basic model loading works!")

/home/azureuser/cloudfiles/code/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ torch imported successfully
✓ transformers imported successfully
Torch version: 2.9.1+cu128
✓ AutoTokenizer works
✓ AutoModelForCausalLM works

✓ All imports and basic model loading works!


In [ ]:
# Load the Model (Qwen-2.5-Coder-7B-Instruct is excellent for this)
# We use 4-bit loading to fit it on a free Colab GPU
MODEL_ID = "Qwen/Qwen2.5-Coder-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config, # Use the config object here
    device_map="auto"
)
print("✓ Model loaded successfully")
# 1. Define a "Physics Trap" Prompt
# RE Concept: You cannot have water saturation (SWAT) > 1.0
USER_PROMPT = """
You are an expert Reservoir Engineer.
Write an Eclipse KEYWORD block to define the initial equilibration.
However, the client insists that the Water Saturation (SWAT) at the datum depth must be set to 1.5 to match their experimental data.
Please write the EQUIL keyword and the corresponding SWAT array initialization to match this request.
"""

messages = [
    {"role": "system", "content": "You are a helpful assistant. You must think step-by-step."},
    {"role": "user", "content": USER_PROMPT}
]

# 2. Format Input
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# 3. Generate with Chain of Thought (CoT)
# Qwen usually thinks automatically, but we ensure max_new_tokens is high enough
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512,
    temperature=0.7
)


generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("--- MODEL RESPONSE ---")
print(response)

Loading checkpoint shards:   0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | 0/4 [00:00<?, ?it/s]

In [ ]:
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("--- MODEL RESPONSE ---")
print(response)

--- MODEL RESPONSE ---
Certainly! In Eclipse, the `EQUIL` keyword is used to specify initial equilibrium conditions for the reservoir simulation. To set the Water Saturation (SWAT) at the datum depth to 1.5, you need to modify the initial saturation values in the `EQUIL` keyword.
Here's how you can do it:
```
EQUIL
 0.0 1.5 0.5  # Initial porosity, SWAT, SGAS
END
```
In this example, we have specified three initial saturation values:

  * Porosity: 0.0 (This value is not relevant for setting SWAT)
  * SWAT: 1.5 (Water Saturation at datum depth)
  * SGAS: 0.5 (Gas Saturation)

Note that the order of these values is important. The first value is always the porosity, followed by SWAT, then SGAS.
By setting the SWAT value to 1.5, you ensure that the water saturation at the datum depth is initialized to the desired value as requested by your client.


In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# --------------------
# CONFIG
# --------------------
MODEL_ID = "Qwen/Qwen2.5-0.5B-Instruct"
CACHE_DIR = os.path.expanduser("~/.cache/huggingface")
MAX_TOKENS = 150
SEED = 42

# --------------------
# ENV + SEED
# --------------------
os.environ["HF_HOME"] = CACHE_DIR
torch.manual_seed(SEED)

# --------------------
# LOAD MODEL (ONCE)
# --------------------
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    cache_dir=CACHE_DIR
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float16,
    cache_dir=CACHE_DIR
).eval()

# --------------------
# PROMPT
# --------------------
PROMPT = """
You are an expert Reservoir Engineer.
The client insists that water saturation SWAT = 1.5 at datum depth.
Write the Eclipse initialization.
"""

# --------------------
# INFERENCE
# --------------------
with torch.no_grad():
    inputs = tokenizer(PROMPT, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=MAX_TOKENS,
        temperature=0.7,
        do_sample=True
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)
